In this notebook 

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns

import plotly.express as px
import cufflinks as cf
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib inline

color_discrete_sequence=["#0015BC", "#DE0100", "#7f7f7f", "#FED105"]
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
cf.go_offline()

Note that we use Twitter_Handles_updated as we found an interesting error in raw data 

In [37]:
df_congress = pd.read_pickle('../data/processed/congress_cleaned_processed.pkl')
twitter_handels = pd.read_csv('../data/processed/Twitter_Handles_updated.csv')

In [38]:
df_congress = pd.merge(df_congress, twitter_handels, how='left',left_on='user_name', right_on='twitter_display_name')

In [39]:
tweet_counts = df_congress.groupby(['Name', 'Party', 'Type']).agg('count')[['text']].reset_index()
tweet_counts.rename(columns={"text": "Total Tweets"}, inplace=True)
tweet_counts.head()

,Name,Party,Type,Total Tweets
0,Abby Finkenauer,Democrat,Representative,2193
1,Abigail Spanberger,Democrat,Representative,2229
2,Adam Kinzinger,Republican,Representative,3182
3,Adam Schiff,Democrat,Representative,4103
4,Adam Smith,Democrat,Representative,2638


# Total number of tweets per party

In [40]:
tweet_count_party = tweet_counts.groupby(['Party', 'Type']).agg('sum').reset_index()
tweet_count_party['Party and Type'] = tweet_count_party['Party'] + ' ' + tweet_count_party['Type']

In [41]:
tweet_count_party

,Party,Type,Total Tweets,Party and Type
0,Democrat,Representative,755224,Democrat Representative
1,Democrat,Senator,281491,Democrat Senator
2,Independent,Senator,17275,Independent Senator
3,Libertarian,Representative,4926,Libertarian Representative
4,Republican,Representative,401690,Republican Representative
5,Republican,Senator,196327,Republican Senator


In [42]:
fig = px.bar(
    tweet_count_party,
    x="Party and Type",
    y="Total Tweets",
    color='Party',
    color_discrete_sequence=["#0015BC", "#7f7f7f", "#FED105", "#DE0100"],
    text='Total Tweets',
    title = 'Total number of tweets per group'
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
# Rotate labels 45 degrees
fig.update_layout(
    xaxis=dict(
        tickangle=-45,
    ),
    yaxis=dict(
        range=[0, 8.5e5]
    ),
    margin=dict(b=5,l=5,r=5,t=40),
    titlefont_size=16,
)
# save for website
fig.write_html(
    file = "../web_app/plotly_files/tweet_barplot.html", 
    full_html = False,
    include_plotlyjs='cdn'
)
fig

# Distribution of tweets

In [46]:
fig = px.histogram(
    tweet_counts, 
    x="Total Tweets", 
    title='(Stacked) Distribution of number of total tweets posted per politicans',
    color='Party',
    color_discrete_sequence=color_discrete_sequence
)
fig.update_layout(
    margin=dict(b=5,l=5,r=5,t=40),
    titlefont_size=16,
)
# save for website
fig.write_html(
    file = "../web_app/plotly_files/tweet_distribution.html", 
    full_html = False,
    include_plotlyjs='cdn'
)
fig

In [45]:
df_congress.head()

,user_id,user_name,id,created_at,text,retweet,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id,Twitter,State,Party,Type,Name,twitter_display_name,length,Party and Type
0,240760644,"Rep. Paul Gosar, DDS",824768889498198016,2017-01-27 00:00:01,good riddance we need leaders who will not und...,False,12,18,NaN,NaN,RepGosar,AZ,Republican,Representative,Paul A. Gosar,"Rep. Paul Gosar, DDS",72,Republican Representative
1,22509548,Darrell Issa,824769012122923008,2017-01-27 00:00:30,rep @darrellissa on reaching across the aisle ...,sdutIdeas,20,0,NaN,NaN,DarrellIssa,CA,Republican,Representative,Darrell Issa,Darrell Issa,62,Republican Representative
2,133028836,Adam Schiff,824769018871500800,2017-01-27 00:00:32,mexico isnt paying for a wall and donald trump...,False,7,21,NaN,NaN,RepAdamSchiff,CA,Democrat,Representative,Adam Schiff,Adam Schiff,106,Democrat Representative
3,1134292500,Rep. Beto O'Rourke,824769057773731840,2017-01-27 00:00:41,why i voted no on hr 7,False,1,9,NaN,NaN,RepBetoORourke,TX,Democrat,Representative,Beto O'Rourke,Rep. Beto O'Rourke,22,Democrat Representative
4,1134292500,Rep. Beto O'Rourke,824769068385267713,2017-01-27 00:00:44,town hall,False,6,9,NaN,NaN,RepBetoORourke,TX,Democrat,Representative,Beto O'Rourke,Rep. Beto O'Rourke,9,Democrat Representative
